In [6]:
import cython
import wwz
import numpy as np

# generate data for a sine wave observed for 10 years with random noise added
# data = np.loadtxt(data_path+"ogle_merged/sxp327.csv", delimiter=",", dtype=float)
x = data[:,0]
x = np.linspace(0, 249, num=250)
period = 3
f=1/period
y = np.sin(x*f*np.pi*2)#+np.random.normal(0,0.5,size=len(x))


# Select the size for the newly generated timestamps (tau), it should be less than the length of the given data.
# This will also limit the minimum possible frequecny
ntau = 5000 #(x[-1]-x[0])/4 # no. linear divisions between first and last time stamps of the time series

# Set the low and high frequency bounds and steps to iterate 
frequency_low = 1/125
frequency_high = 1/2
# frequency_steps = np.logspace(np.log10(frequency_low), np.log10(frequency_high), num=1000)
# frequency_steps = frequency_steps[1]-frequency_steps[0]

# Set the override to False (Setting to True will ignore the low and high frequency limitations)
override = False

# gather the frequency parameters into a list [freq_low, freq_high, freq_step, override]
frequency_parameters = np.logspace(np.log10(frequency_low), np.log10(frequency_high), num=400)#[frequency_low, frequency_high, frequency_steps, override]

# We will then select the decay constant for our analyzing wavelet (should be < 0.2), where c = 1/(2*w^2) 
# The analyzing wavelet decays significantly in a single cycle 2*pi/w, where w = 2*pi*f

# f = 20     # we choose 4 since our signal of interest is at 4 h
# w = 2 * np.pi * f
# decay_constant = 1/(2*w**2)
decay_constant = 1/(8*np.pi**2)
# Finally, we select to wether to run with parallization (recommend True)
parallel = True


# Using the above settings, we will now compute the WWZ/WWA for the full timeseries and removed timeseries
wwt_spec_Nyq = wwt_Nyquist_c(timestamps=x,
                     magnitudes=y,
                     time_divisions=ntau,
                     freq_params=frequency_parameters,
                     decay_constant=decay_constant,
                     method='linear',
                     parallel=parallel)

ModuleNotFoundError: No module named 'wwz'

In [16]:
import cython
import test_c_lib

ImportError: Building module test_c_lib failed: ["distutils.errors.CompileError: command 'gcc' failed with exit status 1\n"]

In [ ]:
import pyximport; pyximport.install()
import helloworld

In [9]:
test.test_func(5,4)

AttributeError: module 'test' has no attribute 'test_func'

In [24]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
print("date and time =", dt_string)

date and time = 29-06-2021_13-19-50


In [ ]:
#https://stackoverflow.com/questions/3927628/how-can-i-profile-python-code-line-by-line
import pprofile
profiler = pprofile.Profile()
with profiler:
    wwt_spec_Nyq = wwt_Nyquist(timestamps=x,
                         magnitudes=y,
                         time_divisions=ntau,
                         freq_params=frequency_parameters,
                         decay_constant=decay_constant,
                         method='linear',
                         parallel=parallel)
# Process profile content: generate a cachegrind file and send it to user.

# You can also write the result to the console:
profiler.print_stats()